# Benchmarking Qmachines with different RB protocols

In [ ]:
from IPython.display import clear_output
# !pip install forest-benchmarking
clear_output()

In [ ]:
from functions import *
from pyquil import get_qc
from pyquil import noise

In [ ]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas() #This line make sure that the progress bars looks natural

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
from csv import writer
import _pickle as cPickle
from datetime import datetime

In [ ]:
k_m = 200 #number of sequences
n_m = 800 #number of shots

# target_qubits = [0]
target_qubits = [41,42]
# target_qubits = [0,1]
n_qubit = len(target_qubits)

# qmachine = get_qc('Aspen-M-3', execution_timeout=200, compiler_timeout=200)
qmachine = get_qc( str(n_qubit) + 'q-qvm')  # You can make any 'nq-qvm'

### Depolarising noise in terms of pauli channels
$$ \Lambda_{dep}(\rho) = (1-3\lambda/4)\rho + \lambda/4 X^{\dagger}\rho X + \lambda/4 Y^{\dagger}\rho Y + \lambda/4 Z^{\dagger}\rho Z $$

$$ \Lambda_{dep}(\rho) = (1-\epsilon)\rho + \frac{\epsilon}{d} I $$

So if we want to have a depolarising noise channel with $\epsilon = 0.0001$, we should define it with taking into account that $p_i = \frac{\epsilon}{4} $. So it gives

$p_i \approx 0.000025 $

In [ ]:
# epsilon = 0.01
# p_xi = epsilon/4
# qmachine.qam.gate_noise=(p_xi,p_xi,p_xi)
# qmachine.qam.gate_noise=(0.005,0.010,0.015)
# qmachine.qam.gate_noise=(0,0,p_xi)

In [ ]:
noisy_machine = qmachine.name
# noisy_machine = 'depolarising_noise' + '_epsilon{}'.format(epsilon)
# noisy_machine = 'pauli_stochastic_noise'
path = os.path.join( os.getcwd(), 'output_storage', noisy_machine)
try:
    os.makedirs( path )
except:
    pass

In [ ]:
ordered_layers = np.arange(1,17,2).astype('int')

ordered_layers_arr = np.array( ordered_layers )
between_layers = np.arange(ordered_layers_arr.min(),ordered_layers_arr.max()+1,1).astype('int')

In [ ]:
bench_protocol_func_dict = {'native_conditional_single_qubit':native_rigetti_single_qubit_packs_generator,
                           'native_conditional_conditional_two_qubits':native_universal_two_qubits_packs_generator,
                           'standard_rb_single_qubit':two_design_single_qubit_packs_generator,
                           'standard_rb_two_qubits':two_design_two_qubits_packs_generator}

In [ ]:
sns.set_theme(context='paper', font= 'Serif', style = 'ticks')

In [ ]:
file_path = os.path.join(path, '{}.csv'.format(qmachine.name))
if os.path.isfile(file_path):
    pass
else:
    with open(file_path, 'w') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(['date', 'target qubits', 'number of sequences', 'number of shots',
                                'protocol name', 'circuit depth', 'average of fidelity', 'error of fidelity'])
        f_object.close()

In [ ]:
def record_this_layer(protocol_name:str, qmachine, depth:int, fidelity_average, fidelity_err):
    List = [datetime.today().strftime('%Y-%m-%d'), target_qubits, k_m, n_m,
            protocol_name, depth, fidelity_average, fidelity_err]
    file_path = os.path.join(path, '{}.csv'.format(qmachine.name))
    with open(file_path, 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(List)
        f_object.close()
    return

In [ ]:
def bench_machine(protocol_name:str, qmachine, target_qubits:list, k_m:int, n_m:int):
    bench_protocol_func = bench_protocol_func_dict[protocol_name]
    avg_fdlty_list = []
    err_fdlty_list = []
    
    for m in tqdm(ordered_layers):
        exps = catch_experiments(qmachine, target_qubits = target_qubits,
                                    protocol_name=protocol_name,
                                    layers_num=m, exp_num=k_m)
#         save_experiment(exps, protocol_name, target_qubits, m, k_m)
        
        response = find_machine_response(qmachine, exps, n_m)
        record_this_layer(protocol_name, qmachine, m, averageOfFidelity(response), stdOfFidelity(response))
        
        avg_fdlty_list.append( averageOfFidelity(response) )
        err_fdlty_list.append( stdOfFidelity(response) )
    return np.array(avg_fdlty_list), np.array(err_fdlty_list)

In [ ]:
def prepare_experiments(protocol_name:str, qmachine, target_qubits:list, k_m:int, n_m:int):
    bench_protocol_func = bench_protocol_func_dict[protocol_name]
    
    for m in tqdm(ordered_layers):
        exps = generate_experiments(qmachine, target_qubits = target_qubits,
                                    circuit_gen_func=bench_protocol_func,
                                    layers_num=m, exp_num=k_m)
        save_experiment(exps, protocol_name, target_qubits, m, k_m)
    return 

# single-qubit RB with conditional prob. on native gates

In [ ]:
prepare_experiments('native_conditional_single_qubit', qmachine, target_qubits, k_m, n_m)

In [ ]:
avg_cond_fdlty_arr, err_cond_fdlty_arr = bench_machine('native_conditional_single_qubit', qmachine, [0], k_m, n_m)

plot_decay(ordered_layers_arr, avg_cond_fdlty_arr, err_cond_fdlty_arr, 'native_conditional_single_qubit')

In [ ]:
avg_cond_fdlty_arr, err_cond_fdlty_arr

# RB with conditional prob. on two qubits

In [ ]:
#Classical compilation process
prepare_experiments('native_conditional_conditional_two_qubits', qmachine, target_qubits, k_m, n_m)

In [ ]:
avg_cond_fdlty_arr, err_cond_fdlty_arr = bench_machine('native_conditional_conditional_two_qubits', qmachine, target_qubits, k_m, n_m)
plot_decay(ordered_layers_arr, avg_cond_fdlty_arr, err_cond_fdlty_arr, ' native_conditional_conditional_two_qubits')

In [ ]:
avg_cond_fdlty_arr

In [ ]:
err_cond_fdlty_arr

## RB standard version single qubit(with Clifford set)

In [ ]:
prepare_experiments('standard_rb_single_qubit', qmachine, target_qubits, k_m, n_m)

In [ ]:
avg_cond_fdlty_arr, err_cond_fdlty_arr = bench_machine('standard_rb_single_qubit', qmachine, target_qubits, k_m, n_m)
plot_decay(ordered_layers_arr, avg_cond_fdlty_arr, err_cond_fdlty_arr, 'standard_rb_single_qubit')

## RB standard version (with Clifford set& non-uniform length gates blocks)

In [ ]:
# prepare_experiments('standard_rb_non_uniform_single_qubit', qmachine, target_qubits, k_m, n_m)

In [ ]:
# avg_cond_fdlty_arr, err_cond_fdlty_arr = bench_machine('standard_rb_non_uniform_single_qubit', qmachine, target_qubits, k_m, n_m)
# plot_decay(ordered_layers_arr, avg_cond_fdlty_arr, err_cond_fdlty_arr, 'standard_rb_non_uniform_single_qubit')

## RB standard version two-qubits (with Clifford set in fixed length circuit blocks)

In [ ]:
prepare_experiments('standard_rb_two_qubits', qmachine, target_qubits, k_m, n_m)

In [ ]:
avg_cond_fdlty_arr, err_cond_fdlty_arr = bench_machine('standard_rb_two_qubits', qmachine, target_qubits, k_m, n_m)
plot_decay(ordered_layers_arr, avg_cond_fdlty_arr, err_cond_fdlty_arr, 'standard_rb_two_qubits')

In [ ]:
avg_cond_fdlty_arr

In [ ]:
err_cond_fdlty_arr

## Diamond distance

In [ ]:
fig = plt.figure(dpi = 800)
ax = fig.add_subplot()

plot_decay(ordered_layers_arr[1:], avg_cond_fdlty_arr[1:], err_cond_fdlty_arr[1:],
           ' conditional_universal_circuit', axes = ax)

plt.xlabel('Depth', fontsize=18)
plt.ylabel('Average of Fidelity', fontsize=16)
plt.title(r'RB versions comparison ($\epsilon = {}$)'.format(epsilon))

plt.legend()

# fig.savefig('RB_comparison.png')

In [ ]:
unitary_to_orthogonal()

r parameter for Native gateset

In [ ]:
decay_param(popt_native[0], n_qubit)

r parameter for Clifford gateset

In [ ]:
decay_param(popt_std[0], n_qubit)

r parameter for Native benchmarking new method!

In [ ]:
decay_param(popt_cond[0], n_qubit)